In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from constantes import html_elements, urls, total_pages, company_name, target_positions

## Crear el webdriver de Chrome

In [2]:
opts = Options()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)

/var/folders/3d/455lhm0d5c11ldvj7pr4mp1h0000gn/T/ipykernel_9914/1088745581.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)


## Funciones

### Funcion para iniciar sesion

In [3]:
def get_login(driver):
    driver.get(urls['home'])

    secret_file = open('secret.txt', 'r')
    lineas = secret_file.readlines()
    user = lineas[0].strip()
    password = lineas[1].strip()
    secret_file.close()

    input_user = driver.find_element(By.XPATH, html_elements['login__input_email'])
    input_pass = driver.find_element(By.XPATH, html_elements['login__input_pass'])

    input_user.send_keys(user)
    input_pass.send_keys(password)

    boton = driver.find_element(By.XPATH, html_elements['login__button'])
    boton.click()

### Funcion para ir a la pagina de empleados de la empresa

In [4]:
def go_to__search_page(driver, page=1):
    search_url__base = urls['searching_people'](page)
    driver.get(search_url__base)

### Funcion para obtener los puestos de trabajo de la empresa

In [5]:
def scraping_page(driver):

    acc = []
    
    for i in range(1, total_pages + 1):
        
        page = i
        go_to__search_page(driver, page) 

        lista_empleados__nombres = driver.find_elements(By.XPATH, html_elements['employees_name'])
        lista_empleados__link_page = driver.find_elements(By.XPATH, html_elements['employees_page'])
        lista_empleados__cargo = driver.find_elements(By.XPATH, html_elements['employees_position'])
        
        for web_nombre,web_cargo,web_link in zip(lista_empleados__nombres, lista_empleados__cargo, lista_empleados__link_page):
            nombre = web_nombre.text 
            cargo = web_cargo.text
            link = web_link.get_attribute("href")
            
            data = { 'nombre': nombre, 'cargo': cargo, 'page': link }
            
            acc.append(data)

    return acc
            

### Clasificacion de puestos de trabajos en funcion de los puestos objetivos

In [6]:
def classify_by_position_target(all_company_positions, target_positions):
    acc = {}
    for position in all_company_positions:
        
        for target in target_positions:
            
            if target in position['cargo'].lower():
                
                if target in acc:
                    acc[target].append(position)
                else:
                    acc[target] = [position]
                
    return acc

### Crear archivos con cargos obtenidos

In [7]:
def creating_files(target_positions, interested_positions):

    for target in target_positions:
        file = open(f"{target}_{company_name}.txt", 'w')
        
        if target in interested_positions:
            position_data = interested_positions[target]

            for p in position_data:

                txt = f"Nombre: {p['nombre']}\nCargo: {p['cargo']}\n link: {p['page']}\n\n"
                file.write(txt)

        file.close()

## Ejecucion de funciones

In [8]:
get_login(driver)

In [9]:
go_to__search_page(driver)

In [10]:
all_positions = scraping_page(driver)

In [11]:
interested_positions = classify_by_position_target(all_positions, target_positions)

In [12]:
creating_files(target_positions, interested_positions)